## **Research Questions**
- How does housing accessibility and rent burden differ across New York City's boroughs?
- How does housing policy and inequality extend to educational inequality in New York City?
- To what extent does racial and socioeconomic segregation in New York City shape its student's educational opportunities and outcomes?

In our project, we chose to look at multiple datasets to see these varying trends and answer different parts of our questions.

## **Data Collection & Cleaning**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import seaborn as sns
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
from sklearn.metrics import r2_score


### Part One: Housing
**Dataset One**: [Housing and Rent Burden](https://data.cityofnewyork.us/City-Government/Demographics-and-profiles-at-the-Neighborhood-Tabu/hyuz-tij8)

First, we loaded out dataset and subsetted the data to only include values of percent of income spent on rent -- measuring burden of rent on a household. Additionally, we removed the last columns that included values for the Staten Island Cemetery. We then did the same operations with gross rent to create two different dataframes to look at. The heads of the two dataframes are displayed below.

In [3]:
# load csv and dropping rows
rent = pd.read_csv("rent.csv", skiprows=5)


# number of rows to drop 
n = 22
  
# dropping last n rows using drop 
rent.drop(rent.tail(n).index, inplace = True) 

# check
# print(rent.tail())

# keep only rent burden rows
rent_burden = rent[142:-2]

# keep only gross rent rows
gross_rent = rent[132:140]

# remove Staten Island Cemetery column
rent_burden = rent_burden.iloc[:,:-4]
gross_rent = gross_rent.iloc[:,:-4]


# check
# display(rent_burden.head())
# display(gross_rent.head())

# print(len(rent_burden), ' ', len(gross_rent))


After looking at these dataframes, we realized have a ton of unnecessary columns. We removed the estimates, margin of error, and percent margin of error columns and only kept the percent column. We performed this by only keeping every fourth column starting with the third column. Then we inserted back the first column which contained our descriptions/labels.

We performed this for both the rent_burden and gross_rent dataframes.

In [4]:
# for rent_burden
# get rid of estimates, MOE and Percent MOE columns - keep only percents
rent_burden_percent = rent_burden.iloc[:,3::4].copy()
# save the row names
rent_burden_row = rent_burden.iloc[:, 0].copy()
# insert first column back into dataframe
rent_burden_percent.insert(0, 'GRAPI', rent_burden_row)

# for gross_rent
# get rid of estimates, MOE and Percent MOE columns - keep only percents
gross_rent_percent = gross_rent.iloc[:,3::4].copy()
# save the row names
gross_rent_row = gross_rent.iloc[:, 0].copy()
# insert first column back into dataframe
gross_rent_percent.insert(0, 'Gross Rent', gross_rent_row)

# check
# display(rent_burden_percent.head())
# display(gross_rent_percent.head())


Next, we further cleaned the data by transposing the dataframes, renaming the columns to easier to read strings, then removed the unnamed column. After these operations, we transposed the dataframe back. We performed this on both the rent_burden and gross_rent dataframe.

In [5]:
# rent_burden
rent_burden_percent_t = rent_burden_percent.T
rent_burden_percent_t.rename(columns = {142 : 'occupied_units', 143 : '<15%', 144 : '15-19.9%', 145 : '20-24.9%', 146 : '25-29.9%',
                                         147 : '30-34.9%', 148 : '>35%', 149 : 'not_computed'}, inplace = True)

rent_burden_percent_t= rent_burden_percent_t.iloc[1:,1:8]


# print(rent_burden_percent_t.head())
# print(rent_burden_estimates_t.tail())

rent_burden_percent = rent_burden_percent_t.T
display(rent_burden_percent.head())
# display(rent_burden_percent.tail())


# gross_rent
gross_rent_percent_t = gross_rent_percent.T
gross_rent_percent_t.rename(columns = {132 : 'occupied_units', 133 : '<200', 134 : '200-299', 135 : '300-499', 136 : '500-749',
                                         137 : '750-999', 138 : '1000-1499', 139 : '>1500'}, inplace = True)

gross_rent_percent_t= gross_rent_percent_t.iloc[1:,1:8]


# print(rent_burden_percent_t.head())
# print(rent_burden_estimates_t.tail())

gross_rent_percent = gross_rent_percent_t.T
# display(gross_rent_percent.head())
# display(gross_rent_percent.tail())

,BK72 Williamsburg.2,BK73 North Side-South Side.2,BK76 Greenpoint.2,BK90 East Williamsburg.2,BK09 Brooklyn Heights-Cobble Hill.2,BK38 DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill.2,BK68 Fort Greene.2,BK69 Clinton Hill.2,BK35 Stuyvesant Heights.2,BK75 Bedford.2,...,SI14 Grasmere-Arrochar-Ft. Wadsworth.2,SI24 Todt Hill-Emerson Hill-Heartland Village-Lighthouse Hill.2,SI36 Old Town-Dongan Hills-South Beach.2,SI45 New Dorp-Midland Beach.2,SI01 Annadale-Huguenot-Prince's Bay-Eltingville.2,SI11 Charleston-Richmond Valley-Tottenville.2,SI25 Oakwood-Oakwood Beach.2,SI32 Rossville-Woodrow.2,SI48 Arden Heights.2,SI54 Great Kills.2
<15%,12.7,12.6,15.5,11.9,20.7,16.7,14.6,13.1,11,10.7,...,13.8,12.9,13.7,17.1,13.9,12.7,12.6,12,13.9,10.7
15-19.9%,5.4,11.6,12.7,13.3,15.8,14.8,13.4,13.4,10.8,11.3,...,13,15,10.5,9.7,17.2,12.3,16.9,20.5,24,21.9
20-24.9%,5.7,13,14.5,14.8,16.2,14.5,13.8,14.1,11.4,9.8,...,10.9,13.8,11.3,10.2,14.8,10.1,10.2,19.5,14.2,14.3
25-29.9%,9.8,10.4,12.2,12.6,13.8,13.6,13.4,11.9,11.4,11.6,...,12,7.8,6.8,9,9.4,9.2,12,4.3,5.4,11.4
30-34.9%,9,10.3,8.7,9,4.2,9.3,10,7.8,10.8,9.9,...,9.9,7.7,14.6,6.8,10.2,12.9,9.4,4.2,0.7,1.3


Next, we transposed both dataframes so that the rows would be the neighborhoods and the columns would be the categories of rent burden/gross rent. Then, we changed the string values in the dataframe to numeric values for both rent_burden_percent_transpose and gross_rent_percent_transpose. 

For rent_burden_percent_transpose, we calculated the sum of the percentage of households that were rent burdened (>30%) for each neighborhood and added a new column that categorized each neighborhood by borough (we created a list of the borough names that matched each column).

For gross_rent_percent_transpose, we created two new columns: one that summed percentage of households that paid high gross rent (greater than 1000) and one for low gross rent (less than 1000).

We created two subsets that only contained the new columns we created.

In [6]:
rent_burden_percent_transpose = rent_burden_percent.T
gross_rent_percent_transpose  = gross_rent_percent.T

rent_burden_percent_transpose['<15%'] = pd.to_numeric(rent_burden_percent_transpose['<15%'])
rent_burden_percent_transpose['15-19.9%'] = pd.to_numeric(rent_burden_percent_transpose['15-19.9%'])
rent_burden_percent_transpose['20-24.9%'] = pd.to_numeric(rent_burden_percent_transpose['20-24.9%'])
rent_burden_percent_transpose['25-29.9%'] = pd.to_numeric(rent_burden_percent_transpose['25-29.9%'])
rent_burden_percent_transpose['30-34.9%'] = pd.to_numeric(rent_burden_percent_transpose['30-34.9%'])
rent_burden_percent_transpose['>35%'] = pd.to_numeric(rent_burden_percent_transpose['>35%'])

gross_rent_percent_transpose['<200'] = pd.to_numeric(gross_rent_percent_transpose['<200'])
gross_rent_percent_transpose['200-299'] = pd.to_numeric(gross_rent_percent_transpose['200-299'])
gross_rent_percent_transpose['300-499'] = pd.to_numeric(gross_rent_percent_transpose['300-499'])
gross_rent_percent_transpose['500-749'] = pd.to_numeric(gross_rent_percent_transpose['500-749'])
gross_rent_percent_transpose['750-999'] = pd.to_numeric(gross_rent_percent_transpose['750-999'])
gross_rent_percent_transpose['>1500'] = pd.to_numeric(gross_rent_percent_transpose['>1500'])

borough = []
for col in rent_burden_percent:
    if col.startswith('BK'):
        borough.append('Brooklyn')
    if col.startswith('BX'):
        borough.append('Bronx')
    if col.startswith('MN'):
        borough.append('Manhattan')
    if col.startswith('QN'):
        borough.append('Queens')
    if col.startswith('SI'):
        borough.append('Staten Island')

rent_burden_percent_transpose['rent_burdened'] = (rent_burden_percent_transpose[['30-34.9%','>35%']]).sum(axis=1)
rent_burden_percent_transpose['borough'] = borough

gross_rent_percent_transpose['low_rent'] = (gross_rent_percent_transpose[['<200','200-299', '300-499', '750-999']]).sum(axis=1)
gross_rent_percent_transpose['high_rent'] = (gross_rent_percent_transpose[['1000-1499','>1500']]).sum(axis=1)

rent_burden_percent_sub = rent_burden_percent_transpose.iloc[:,6:]
# display(rent_burden_percent_sub.head())

gross_rent_percent_sub = gross_rent_percent_transpose.iloc[:,7:]
# display(gross_rent_percent_sub.head())

display(rent_burden_percent_transpose.tail())
display(gross_rent_percent_transpose.head())

,<15%,15-19.9%,20-24.9%,25-29.9%,30-34.9%,>35%,rent_burdened,borough
SI11 Charleston-Richmond Valley-Tottenville.2,12.7,12.3,10.1,9.2,12.9,42.8,55.7,Staten Island
SI25 Oakwood-Oakwood Beach.2,12.6,16.9,10.2,12.0,9.4,39.0,48.4,Staten Island
SI32 Rossville-Woodrow.2,12.0,20.5,19.5,4.3,4.2,39.4,43.6,Staten Island
SI48 Arden Heights.2,13.9,24.0,14.2,5.4,0.7,41.8,42.5,Staten Island
SI54 Great Kills.2,10.7,21.9,14.3,11.4,1.3,40.4,41.7,Staten Island


,<200,200-299,300-499,500-749,750-999,1000-1499,>1500,low_rent,high_rent
BK72 Williamsburg.2,3.8,13.8,15.4,14.3,14.0,23.9,14.8,47.0,14.8
BK73 North Side-South Side.2,1.5,6.3,6.8,12.3,12.8,16.9,43.4,27.4,43.4
BK76 Greenpoint.2,0.6,1.3,2.6,5.5,15.1,32.9,41.9,19.6,41.9
BK90 East Williamsburg.2,0.7,7.1,5.7,14.5,15.7,22.1,34.3,29.2,34.3
BK09 Brooklyn Heights-Cobble Hill.2,0.2,1.4,2.1,5.3,8.0,18.5,64.6,11.7,64.6


After transposing the dataframes, we reset the index of the dataframes to create a neighborhood column. Then we merged rent_burden_percent_sub and gross_rent_percent_sub. After looking at the dataset, we found missing values for Rikers Island (a prison), Queens Airport, and Manhattan Cemetery so we dropped these rows by only keeping the rows where rent burdened was at least 1.

In [7]:
rent_burden_percent_sub.reset_index(inplace=True)
#rent_burden_percent_sub = rent_burden_percent_sub.rename(columns = {'Neighborhood'})

gross_rent_percent_sub.reset_index(inplace=True)
#gross_rent_percent_sub = gross_rent_percent_sub.rename(columns = {'Neighborhood'})

burden_gross = rent_burden_percent_sub.merge(gross_rent_percent_sub)
burden_gross = burden_gross.rename(columns = {'index':'neighborhood'})

# i = burden_gross[((burden_gross.rent_burdened <1)& (burden_gross.low_rent<1) & (burden_gross.high_rent<1))].index
# burden_gross.drop(i)

# Drop Rikers Island, Queens Airport, and Manhattan Cemetery Rows
burden_gross = burden_gross[burden_gross['rent_burdened']>1]

display(burden_gross.head())

,neighborhood,rent_burdened,borough,low_rent,high_rent
0,BK72 Williamsburg.2,66.4,Brooklyn,47.0,14.8
1,BK73 North Side-South Side.2,52.3,Brooklyn,27.4,43.4
2,BK76 Greenpoint.2,45.1,Brooklyn,19.6,41.9
3,BK90 East Williamsburg.2,47.4,Brooklyn,29.2,34.3
4,BK09 Brooklyn Heights-Cobble Hill.2,33.6,Brooklyn,11.7,64.6


In [9]:
rent_burden_percent['brooklyn'] = rent_burden_percent.loc[:,rent_burden_percent.columns.str.startswith('BK')].median(axis=1)
# print(rent_burden_estimates.loc[:,rent_burden_estimates.columns.str.startswith('BK')])
# print(rent_burden_percent['brooklyn'])


rent_burden_percent['bronx'] = rent_burden_percent.loc[:,rent_burden_percent.columns.str.startswith('BX')].median(axis=1)
# print(rent_burden_estimates.loc[:,rent_burden_estimates.columns.str.startswith('BX')])
# print(rent_burden_estimates['bronx'])

rent_burden_percent['manhattan'] = rent_burden_percent.loc[:,rent_burden_percent.columns.str.startswith('MN')].median(axis=1)
# print(rent_burden_estimates.loc[:,rent_burden_estimates.columns.str.startswith('MN')])
# print(rent_burden_estimates['manhattan'])

rent_burden_percent['queens'] = rent_burden_percent.loc[:,rent_burden_percent.columns.str.startswith('QN')].median(axis=1)
# print(rent_burden_estimates.loc[:,rent_burden_estimates.columns.str.startswith('QN')])
# print(rent_burden_estimates['queens'])

rent_burden_percent['staten'] = rent_burden_percent.loc[:,rent_burden_percent.columns.str.startswith('SI')].median(axis=1)
# print(rent_burden_estimates.loc[:,rent_burden_estimates.columns.str.startswith('SI')])
# print(rent_burden_estimates['staten'])

Lastly, we prepare our dataframe to merge with subsequent dataframes by indicating the community district of each of the boroughs manually.

In [10]:
burden_gross.head()
community_district = ['BK 01', 'BK 01','BK 01', 'BK 01','BK 02', 'BK 02', 'BK 02', 'BK 02','BK 03','BK 03', 'BK 04', 'BK 04', 'BK 04',
                     'BK 05', 'BK 05', 'BK 05','BK 06','BK 06','BK 07', 'BK 07', 'BK 07', 'BK 08', 'BK 08', 'BK 09','BK 09',
                     'BK 10','BK 10','BK 11', 'BK 11','BK 11','BK 12', 'BK 12', 'BK 12','BK 13', 'BK 13', 'BK 13', 'BK 13',
                     'BK 14','BK 14','BK 15', 'BK 15', 'BK 15', 'BK 16', 'BK 16', 'BK 16','BK 17', 'BK 17', 'BK 17',
                     'BK 18', 'BK 18', 'BK 18',
                     'BX 02', 'BX 02','BX 01', 'BX 01','BX 03', 'BX 06', 'BX 06', 'BX 03', 'BX 03','BX 04','BX 04','BX 04',
                     'BX 05','BX 05','BX 05','BX 07', 'BX 07','BX 07', 'BX 08', 'BX 08','BX 08','BX 09','BX 09','BX 09','BX 09','BX 09',
                     'BX 10', 'BX 10','BX 10','BX 11','BX 11','BX 11','BX 11','BX 12','BX 12','BX 12','BX 12',
                     'MN 02','MN 02','MN 01','MN 03','MN 03','MN 03','MN 04', 'MN 04', 'MN 05','MN 06','MN 06','MN 06','MN 06',
                     'MN 07','MN 07', 'MN 08','MN 08','MN 08', 'MN 09','MN 09','MN 09','MN 10','MN 10', 'MN 11','MN 11',
                     'MN 12','MN 12','MN 12',
                     'QN 01','QN 01','QN 01','QN 01','QN 01','QN 02','QN 02','QN 02','QN 03','QN 03','QN 03','QN 04', 'QN 04',
                     'QN 05','QN 05','QN 05','QN 05','QN 06','QN 06','QN 07','QN 07','QN 07','QN 07','QN 07','QN 07','QN 07', 'QN 08',
                     'QN 08','QN 08','QN 08','QN 08','QN 09','QN 09','QN 09','QN 10','QN 10','QN 10', 'QN 11','QN 11','QN 11','QN 11',
                     'QN 12','QN 12','QN 12','QN 12','QN 12','QN 12','QN 13','QN 13','QN 13','QN 13','QN 13','QN 13','QN 13',
                     'QN 14','QN 14','QN 14',
                     'SI 01','SI 01','SI 01','SI 01','SI 01','SI 01','SI 01','SI 02','SI 02','SI 02','SI 02','SI 02',
                     'SI 03','SI 03','SI 03','SI 03','SI 03','SI 03']
burden_gross['district'] = community_district
display(burden_gross)

,neighborhood,rent_burdened,borough,low_rent,high_rent,district
0,BK72 Williamsburg.2,66.4,Brooklyn,47.0,14.8,BK 01
1,BK73 North Side-South Side.2,52.3,Brooklyn,27.4,43.4,BK 01
2,BK76 Greenpoint.2,45.1,Brooklyn,19.6,41.9,BK 01
3,BK90 East Williamsburg.2,47.4,Brooklyn,29.2,34.3,BK 01
4,BK09 Brooklyn Heights-Cobble Hill.2,33.6,Brooklyn,11.7,64.6,BK 02
...,...,...,...,...,...,...
189,SI11 Charleston-Richmond Valley-Tottenville.2,55.7,Staten Island,12.7,40.8,SI 03
190,SI25 Oakwood-Oakwood Beach.2,48.4,Staten Island,22.8,22.4,SI 03
191,SI32 Rossville-Woodrow.2,43.6,Staten Island,18.2,27.9,SI 03
192,SI48 Arden Heights.2,42.5,Staten Island,19.9,29.6,SI 03


Finally, we have a dataset comprised of two columns -- district and rent_burdened to be able to be merge with other datasets.

In [10]:
housing = burden_gross.groupby(['district'])['rent_burdened','low_rent','high_rent'].mean().reset_index()

housing = housing.sort_values('district')

housing.reset_index(inplace=True)
housing = housing.drop(columns=['index'])

housing.iloc[:,1:] = housing.iloc[:,1:].round(2)
display(housing[0:20])

<ipython-input-10-98ea94a62efb>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  housing = burden_gross.groupby(['district'])['rent_burdened','low_rent','high_rent'].mean().reset_index()


,district,rent_burdened,low_rent,high_rent
0,BK 01,52.80,30.80,33.60
1,BK 02,41.55,25.08,47.02
2,BK 03,56.05,35.65,18.40
3,BK 04,56.73,18.90,38.93
4,BK 05,57.53,35.87,14.73
5,BK 06,40.70,17.40,57.00
6,BK 07,52.90,22.13,29.00
7,BK 08,48.25,25.80,34.15
8,BK 09,58.40,33.55,11.80
9,BK 10,52.70,19.85,24.55


We realized that we wanted to include demographic data, but we could not find any demographic datasets of New York City with the parameters we needed. The closest data we found individually described each of the community districts so we made the decision to manually add the rows in ourselves. The data that we used can be found at the [NYC Planning Community District Profiles](https://communityprofiles.planning.nyc.gov/about).

In [11]:
minority = [37, 53.6, 74, 81.1, 96.3, 35.9, 75.6, 77.7, 77.2, 45.4, 59.8, 28.9, 44.1, 59.2, 34.5, 97.4, 97.3, 77.3,
           98, 98, 96, 98, 98.9, 96, 94.2, 67.7, 97.1, 72.5, 78.3, 93.7,
           27.8, 27.8, 66.7, 40.9, 40.9, 30.3, 32, 24.7, 72.4, 85.8, 86.3, 81.1,
           51.7, 71.1, 89.6, 93.8, 48.3, 52.3, 74.6, 70.1, 82, 78.5, 60.6, 98.2, 89.1, 65.8, 
           62.3, 33.9, 17.1]

housing['minority_rate']= minority

### Part Two: Poverty
**Dataset Two**: [Poverty and Indicators](https://data.cityofnewyork.us/City-Government/NYCgov-Poverty-Measure-Data-2018-/hveh-vctc).


First, we load in our poverty datase.

In [12]:
poverty = pd.read_csv("poverty.csv")
display(poverty.head())

,Geography,Name,Level,Poverty_Rate_2017
0,NYC,New York City,City,18.0%
1,BX,The Bronx,Boro,28.0%
2,BK,Brooklyn,Boro,19.8%
3,MN,Manhattan,Boro,16.2%
4,QN,Queens,Boro,12.1%


Next, we drop the summary rows and sort alphabetically by borough to match our previous dataset. Then, we reset our index.

In [13]:
poverty = poverty.iloc[6:]

poverty = poverty.sort_values('Geography')

poverty.reset_index(inplace=True)

poverty = poverty.drop(columns=['index', 'Level'])
display(poverty[0:20])

,Geography,Name,Poverty_Rate_2017
0,BK 01,Greenpoint/Williamsburg,23.3%
1,BK 02,Fort Greene/Brooklyn Heights,14.9%
2,BK 03,Bedford Stuyvesant,24.4%
3,BK 04,Bushwick,27.1%
4,BK 05,East New York/Starrett City,24.7%
5,BK 06,Park Slope/Carroll Gardens,7.8%
6,BK 07,Sunset Park,22.1%
7,BK 08,Crown Heights/Prospect Heights,20.0%
8,BK 09,South Crown Heights/Lefferts Gardens,19.1%
9,BK 10,Bay Ridge/Dyker Heights,15.2%


Finally, we added another column that indicated the overall borough the district was in.

In [14]:
borough = []
for col in poverty['Geography']:
    if col.startswith('BK'):
        borough.append('Brooklyn')
    if col.startswith('BX'):
        borough.append('Bronx')
    if col.startswith('MN'):
        borough.append('Manhattan')
    if col.startswith('QN'):
        borough.append('Queens')
    if col.startswith('SI'):
        borough.append('Staten Island')
poverty['borough'] = borough

Next, we renamed our columns to have the same syntax as our housing dataframe for uniformity, and changed the poverty rate column from strings to floats.

In [15]:
poverty = poverty.rename(columns = {'Geography':'district', 'Name':'neighborhood','Poverty_Rate_2017':'poverty_rate'})

poverty['poverty_rate'] = poverty['poverty_rate'].str.rstrip('%').astype('float')

display(poverty.head())

,district,neighborhood,poverty_rate,borough
0,BK 01,Greenpoint/Williamsburg,23.3,Brooklyn
1,BK 02,Fort Greene/Brooklyn Heights,14.9,Brooklyn
2,BK 03,Bedford Stuyvesant,24.4,Brooklyn
3,BK 04,Bushwick,27.1,Brooklyn
4,BK 05,East New York/Starrett City,24.7,Brooklyn


**Dataset Three**: [Population](https://data.cityofnewyork.us/City-Government/New-York-City-Population-By-Community-Districts/xi7c-iiu2)

We wanted to get a good sense of the magnitude of poverty levels in each of the districts, so we also decided to look at population levels in each of the community districts. In order to clean it to make it mergable with our poverty dataframe, we create a new column with the same community district names and sorted our datframe alphabetically by this column


In [16]:
population = pd.read_csv("population.csv")
borough1 = ['BX 01','BX 02','BX 03', 'BX 04', 'BX 05', 'BX 06', 'BX 07', 'BX 08', 'BX 09', 'BX 10', 'BX 11', 'BX 12',
          'BK 01', 'BK 02', 'BK 03', 'BK 04' ,'BK 05', 'BK 06', 'BK 07', 'BK 08', 'BK 09', 'BK 10', 'BK 11', 'BK 12',
          'BK 13', 'BK 14', 'BK 15', 'BK 16', 'BK 17', 'BK 18',
          'MN 01', 'MN 02', 'MN 03', 'MN 04', 'MN 05','MN 06', 'MN 07', 'MN 08', 'MN 09', 'MN 10', 'MN 11', 'MN 12',
          'QN 01', 'QN 02', 'QN 03', 'QN 04', 'QN 05','QN 06','QN 07','QN 08','QN 09', 'QN 10','QN 11','QN 12','QN 13','QN 14',
          'SI 01', 'SI 02', 'SI 03']
population['district'] = borough1
population = population.sort_values('district')
population.reset_index(inplace=True)

population = population.drop(columns=['index'])
display(population.head())

,Borough,CD Number,CD Name,1970 Population,1980 Population,1990 Population,2000 Population,2010 Population,district
0,Brooklyn,1,"Williamsburg, Greenpoint",179390,142942,155972,160338,173083,BK 01
1,Brooklyn,2,"Brooklyn Heights, Fort Greene",110221,92732,94534,98620,99617,BK 02
2,Brooklyn,3,Bedford Stuyvesant,203380,133379,138696,143867,152985,BK 03
3,Brooklyn,4,Bushwick,137902,92497,102572,104358,112634,BK 04
4,Brooklyn,5,"East New York, Starrett City",170791,154931,161350,173198,182896,BK 05


We then added the 2010 (latest estimate) poulation of each of the community districts to our poverty dataframe.

In [17]:
poverty['population'] = population['2010 Population']
poverty.head()

,district,neighborhood,poverty_rate,borough,population
0,BK 01,Greenpoint/Williamsburg,23.3,Brooklyn,173083
1,BK 02,Fort Greene/Brooklyn Heights,14.9,Brooklyn,99617
2,BK 03,Bedford Stuyvesant,24.4,Brooklyn,152985
3,BK 04,Bushwick,27.1,Brooklyn,112634
4,BK 05,East New York/Starrett City,24.7,Brooklyn,182896


**Combining Housing and Poverty**

In [18]:
housing_poverty = poverty.merge(housing, on = 'district')
display(housing_poverty[0:15])

,district,neighborhood,poverty_rate,borough,population,rent_burdened,low_rent,high_rent,minority_rate
0,BK 01,Greenpoint/Williamsburg,23.3,Brooklyn,173083,52.80,30.80,33.60,37.0
1,BK 02,Fort Greene/Brooklyn Heights,14.9,Brooklyn,99617,41.55,25.08,47.02,53.6
2,BK 03,Bedford Stuyvesant,24.4,Brooklyn,152985,56.05,35.65,18.40,74.0
3,BK 04,Bushwick,27.1,Brooklyn,112634,56.73,18.90,38.93,81.1
4,BK 05,East New York/Starrett City,24.7,Brooklyn,182896,57.53,35.87,14.73,96.3
5,BK 06,Park Slope/Carroll Gardens,7.8,Brooklyn,104709,40.70,17.40,57.00,35.9
6,BK 07,Sunset Park,22.1,Brooklyn,126230,52.90,22.13,29.00,75.6
7,BK 08,Crown Heights/Prospect Heights,20.0,Brooklyn,96317,48.25,25.80,34.15,77.7
8,BK 09,South Crown Heights/Lefferts Gardens,19.1,Brooklyn,98429,58.40,33.55,11.80,77.2
9,BK 10,Bay Ridge/Dyker Heights,15.2,Brooklyn,124491,52.70,19.85,24.55,45.4


Finally, we export our completed data on housing and poverty to a csv file.

In [19]:
housing_poverty.to_csv('housing_poverty.csv')

### Part Three: Education

In [25]:
## load data
school_data = pd.read_csv("Ny_School.csv")
housing_data = pd.read_csv("Student_Temp_Housing.csv")
quality_data = pd.read_csv("schoolquality.csv")

#### Renaming Columns
In this section, we renamed the columns by making them lower_case and replacing spaces in order to make them easier to work with.

In [26]:
new_colnames = []
for col in school_data.columns:
    new_colnames.append(col.lower())
    
new_colnames_2 = []
for col in housing_data.columns:
    new_colnames_2.append(col.lower())
    for col in range(len(new_colnames)):
        new_colnames[col] = new_colnames[col].replace(" ", "_")
# print (new_colnames)

new_colnames_3 = []
for col in quality_data.columns:
    new_colnames_3.append(col.lower())
    for col in range(len(new_colnames_3)):
        new_colnames_3[col] = new_colnames_3[col].replace(" ", "_")
        new_colnames_3[col] = new_colnames_3[col].replace("percent", "%")
# print (new_colnames_3)

sdata = school_data.copy()
sdata.columns = new_colnames

for col in range(len(new_colnames_2)):
    new_colnames_2[col] = new_colnames_2[col].replace(" ", "_")
# print (new_colnames_2)

sdata = school_data.copy()
sdata.columns = new_colnames

hdata = housing_data.copy()
hdata.columns = new_colnames_2

qdata = quality_data.copy()
qdata.columns = new_colnames_3

#### Dropping Columns
Then, we dropped the columns we did not need, only keeping ones relative to our research question which involved keeping school demographic (race make-up), poverty, and students in temporary housing data.

In [27]:
sdata.drop(sdata.loc[:,["total_enrollment", "#_female", "%_female", "#_male", "%_male", "#_black", "#_white", "#_asian", "#_hispanic", "#_multiple_race_categories_not_represented", "#_students_with_disabilities", "#_english_language_learners", "%_students_with_disabilities", "#_poverty", "%_multiple_race_categories_not_represented", "%_english_language_learners", "economic_need_index"]], inplace = True, axis = 1)
sdata.drop(sdata.loc[:,["grade_pk_(half_day_&_full_day)", 'grade_k', 'grade_1', 'grade_2', 'grade_3', 'grade_4', 'grade_5', 'grade_6', 'grade_7', 'grade_8', 'grade_9', 'grade_10', 'grade_11', 'grade_12']], inplace = True, axis = 1)
# display(sdata.head())

hdata.drop(hdata.loc[:, ['#_doubled_up', "school_name", '#_residing_in_non-dhs_shelter', '#_doubled_up'  ]], inplace = True, axis = 1)
# display(hdata.head())

qdata.drop(qdata.loc[:,["school_name", "school_type", 'rigorous_instruction_rating', 'collaborative_teachers_rating', 'supportive_environment_rating', 'effective_school_leadership_rating', 'strong_family-community_ties_rating', 'trust_rating', 'student_achievement_rating', 'quality_review_-_how_interesting_and_challenging_is_the_curriculum?', 'quality_review_-_how_effective_is_the_teaching_and_learning?', 'quality_review_-_how_well_does_the_school_assess_what_students_are_learning?', 'quality_review_-_how_clearly_are_high_expectations_communicated_to_students_and_staff?', 'quality_review_-_how_well_do_teachers_work_with_each_other?', 'quality_review_-_how_safe_and_inclusive_is_the_school_while_supporting_social-emotional_growth?','quality_review_-_how_well_does_the_school_allocate_and_manage_resources?', 'quality_review_-_how_well_does_the_school_identify,_track,_and_meet_its_goals?']], inplace = True, axis = 1)
qdata.drop(qdata.loc[:,['quality_review_-_how_thoughtful_is_the_school’s_approach_to_teacher_development_and_evaluation?', 'quality_review_-_how_well_are_school_decisions_evaluated_and_adjusted?', 'quality_review_-_dates_of_review', '%_self-contained', '%_asian', '%_black', '%_hispanic', '%_white', 'years_of_principal_experience_at_this_school', '%_of_teachers_with_3_or_more_years_of_experience', 'student_attendance_rate', 'teacher_attendance_rate', '%_in_temp_housing']], inplace = True, axis = 1)
display(qdata.head())

,dbn,enrollment,rigorous_instruction_-_%_positive,collaborative_teachers_-_%_positive,supportive_environment_-_%_positive,effective_school_leadership_-_%_positive,strong_family-community_ties_-_%_positive,trust_-_%_positive,average_incoming_ela_proficiency_(based_on_4th_grade),average_incoming_math_proficiency_(based_on_4th_grade),%_english_language_learners,%_students_with_disabilities,economic_need_index,%_hra_eligible,%_of_students_chronically_absent
0,01M015,161,0.91,0.97,0.89,0.99,0.94,0.96,NaN,NaN,0.075,0.304,0.895,0.702,0.215
1,01M019,247,0.91,0.91,0.92,0.91,0.93,0.93,NaN,NaN,0.036,0.340,0.605,0.381,0.324
2,01M020,499,0.67,0.77,0.68,0.83,0.91,0.85,NaN,NaN,0.186,0.224,0.699,0.489,0.256
3,01M034,337,0.74,0.71,0.64,0.75,0.90,0.81,NaN,NaN,0.080,0.380,0.872,0.677,0.305
4,01M063,178,0.80,0.88,0.86,0.92,0.92,0.93,NaN,NaN,0.028,0.354,0.709,0.500,0.251


#### Adding Columns and Grouping
In this section, we added two new columns in borough + district and boroughs so that we could group by both. In order to achieve this, we wrote two functions which took the dbn school number and converted them to specific boroughs and districts. Then, we grouped all the data by the boroughs and districts, replaced "suppressed values" with 0, and rounded and converted the decimals to percentages.

In [28]:
def label_borough(col):
    district_number = col['dbn'][0:2]
    if col['dbn'].find("M") == 2:
      return 'Manhattan ' + district_number
    if col['dbn'].find("K") == 2:
      return 'Brooklyn ' + district_number
    if col['dbn'].find("X") == 2 :
      return 'Bronx ' + district_number
    if col['dbn'].find("Q") == 2:
      return 'Queens ' + district_number
    if col['dbn'].find("R") == 2:
      return 'Staten Island ' + district_number

def labelBorough(col):
    if col['borough_and_district_#'].find("Manhattan") != -1:
      return 'Manhattan'
    if col['borough_and_district_#'].find("Brooklyn") != -1:
      return 'Brooklyn'
    if col['borough_and_district_#'].find("Bronx") != -1:
      return 'Bronx'
    if col['borough_and_district_#'].find("Queens") != -1:
      return 'Queens'
    if col['borough_and_district_#'].find("Staten Island")!= -1:
      return 'Staten Island'


sdata['borough_and_district_#'] = sdata.apply (lambda col: label_borough(col), axis=1)

sdata = sdata.groupby(['borough_and_district_#'], as_index=False).mean()

sdata.iloc[:,1:6] = sdata.iloc[:,1:6].mul(100).round(2)
display(sdata.head())

hdata = hdata.replace("s", 0)

hdata['borough_and_district_#'] = hdata.apply (lambda col: label_borough(col), axis=1)

hdata["%_students_in_temporary_housing"] = pd.to_numeric(hdata["%_students_in_temporary_housing"])
hdata["#_total_students"] = pd.to_numeric(hdata["#_total_students"])
hdata["#_students_residing_in_shelter"] = pd.to_numeric(hdata["#_students_residing_in_shelter"])

hdata = hdata.groupby(['borough_and_district_#'], as_index=False).mean()
hdata["borough"] = hdata.apply(lambda col: labelBorough(col), axis=1)
hdata.iloc[:,2] = hdata.iloc[:,2].mul(100).round(2)
hdata.iloc[: ,1:4] = hdata.iloc[: ,1:4].round(2)

display(hdata.head())

qdata['borough_and_district_#'] = qdata.apply (lambda col: label_borough(col), axis=1)

qdata = qdata.groupby(['borough_and_district_#'], as_index=False).mean()

qdata.iloc[:,2:] = qdata.iloc[:,2:].mul(100).round(2)
display(qdata.head())


,borough_and_district_#,%_asian,%_black,%_hispanic,%_white,%_poverty
0,Bronx 07,1.10,26.60,69.87,1.42,91.76
1,Bronx 08,5.04,24.28,64.12,5.17,81.42
2,Bronx 09,1.50,28.60,67.84,1.24,90.82
3,Bronx 10,4.41,17.52,72.22,4.41,85.26
4,Bronx 11,6.76,40.91,42.18,8.06,78.04


,borough_and_district_#,#_total_students,%_students_in_temporary_housing,#_students_residing_in_shelter,borough
0,Bronx 07,458.76,18.00,34.31,Bronx
1,Bronx 08,508.66,14.73,31.05,Bronx
2,Bronx 09,515.24,21.07,43.58,Bronx
3,Bronx 10,644.69,18.66,28.36,Bronx
4,Bronx 11,643.30,11.88,25.68,Bronx


,borough_and_district_#,enrollment,rigorous_instruction_-_%_positive,collaborative_teachers_-_%_positive,supportive_environment_-_%_positive,effective_school_leadership_-_%_positive,strong_family-community_ties_-_%_positive,trust_-_%_positive,average_incoming_ela_proficiency_(based_on_4th_grade),average_incoming_math_proficiency_(based_on_4th_grade),%_english_language_learners,%_students_with_disabilities,economic_need_index,%_hra_eligible,%_of_students_chronically_absent
0,Bronx 07,415.740741,80.15,82.19,74.70,84.89,88.89,87.67,240.50,237.10,19.32,26.71,88.79,72.15,33.05
1,Bronx 08,518.918919,78.81,82.14,75.84,82.22,87.19,87.54,254.13,249.31,14.33,26.26,76.42,60.84,32.56
2,Bronx 09,479.442308,79.35,82.54,75.19,83.92,88.10,88.08,240.58,234.50,24.39,23.99,87.59,70.51,30.81
3,Bronx 10,632.913793,81.95,84.97,77.86,86.43,89.29,89.91,248.11,251.37,22.87,22.48,81.74,63.90,25.16
4,Bronx 11,653.069767,80.23,84.09,77.28,85.16,88.86,87.98,253.20,248.00,9.78,21.48,64.25,49.76,26.14


#### Merging the two datasets
For this step, we created the final dataframe by merging both the education and housing data by borough and district number using the merge method.

In [29]:
one_ds = pd.merge(sdata, hdata, on = "borough_and_district_#", how = "inner")
final_ds = pd.merge(one_ds, qdata, on = "borough_and_district_#", how = "inner")
final_ds.head()

,borough_and_district_#,%_asian,%_black,%_hispanic,%_white,%_poverty,#_total_students,%_students_in_temporary_housing,#_students_residing_in_shelter,borough,...,effective_school_leadership_-_%_positive,strong_family-community_ties_-_%_positive,trust_-_%_positive,average_incoming_ela_proficiency_(based_on_4th_grade),average_incoming_math_proficiency_(based_on_4th_grade),%_english_language_learners,%_students_with_disabilities,economic_need_index,%_hra_eligible,%_of_students_chronically_absent
0,Bronx 07,1.10,26.60,69.87,1.42,91.76,458.76,18.00,34.31,Bronx,...,84.89,88.89,87.67,240.50,237.10,19.32,26.71,88.79,72.15,33.05
1,Bronx 08,5.04,24.28,64.12,5.17,81.42,508.66,14.73,31.05,Bronx,...,82.22,87.19,87.54,254.13,249.31,14.33,26.26,76.42,60.84,32.56
2,Bronx 09,1.50,28.60,67.84,1.24,90.82,515.24,21.07,43.58,Bronx,...,83.92,88.10,88.08,240.58,234.50,24.39,23.99,87.59,70.51,30.81
3,Bronx 10,4.41,17.52,72.22,4.41,85.26,644.69,18.66,28.36,Bronx,...,86.43,89.29,89.91,248.11,251.37,22.87,22.48,81.74,63.90,25.16
4,Bronx 11,6.76,40.91,42.18,8.06,78.04,643.30,11.88,25.68,Bronx,...,85.16,88.86,87.98,253.20,248.00,9.78,21.48,64.25,49.76,26.14


We then created a column that indicated overall percentage of underrepresented minorities.

In [31]:
final_ds['%_minority']= (final_ds[['%_black','%_hispanic']]).sum(axis=1)
# final_ds = final_ds.drop(columns=['index'])

Finally, we created a new column that indicated the overall borough a row was in.

In [33]:
borough = []
for col in final_ds['borough_and_district_#']:
    if col.startswith('Bronx'):
        borough.append('Bronx')
    if col.startswith('Brooklyn'):
        borough.append('Brooklyn')
    if col.startswith('Manhattan'):
        borough.append('Manhattan')
    if col.startswith('Queens'):
        borough.append('Queens')
    if col.startswith('Staten'):
        borough.append('Staten Island')
final_ds['borough'] = borough

,borough_and_district_#,%_asian,%_black,%_hispanic,%_white,%_poverty,#_total_students,%_students_in_temporary_housing,#_students_residing_in_shelter,borough,...,strong_family-community_ties_-_%_positive,trust_-_%_positive,average_incoming_ela_proficiency_(based_on_4th_grade),average_incoming_math_proficiency_(based_on_4th_grade),%_english_language_learners,%_students_with_disabilities,economic_need_index,%_hra_eligible,%_of_students_chronically_absent,%_minority
0,Bronx 07,1.10,26.60,69.87,1.42,91.76,458.76,18.00,34.31,Bronx,...,88.89,87.67,240.50,237.10,19.32,26.71,88.79,72.15,33.05,96.47
1,Bronx 08,5.04,24.28,64.12,5.17,81.42,508.66,14.73,31.05,Bronx,...,87.19,87.54,254.13,249.31,14.33,26.26,76.42,60.84,32.56,88.40
2,Bronx 09,1.50,28.60,67.84,1.24,90.82,515.24,21.07,43.58,Bronx,...,88.10,88.08,240.58,234.50,24.39,23.99,87.59,70.51,30.81,96.44
3,Bronx 10,4.41,17.52,72.22,4.41,85.26,644.69,18.66,28.36,Bronx,...,89.29,89.91,248.11,251.37,22.87,22.48,81.74,63.90,25.16,89.74
4,Bronx 11,6.76,40.91,42.18,8.06,78.04,643.30,11.88,25.68,Bronx,...,88.86,87.98,253.20,248.00,9.78,21.48,64.25,49.76,26.14,83.09


Finally, we exported the cleaned dataframe to a csv file.

In [34]:
final_ds.to_csv('final_ds.csv')

## **Data Description and Limitations**

### Dataset One: [Housing](https://data.cityofnewyork.us/City-Government/Demographics-and-profiles-at-the-Neighborhood-Tabu/hyuz-tij8)

##### Description
* The original dataset had different rows be the housing characteristics from rent, household type, rent burden, etc… and the columns were the different neighborhoods. The cleaned data set had the rows be the percent burden of rent and the columns be the five boroughs in NYC.
* This dataset was created as a part of an initiative to increase transparency, accessibility, and accountability of the City government.
* According to their website, NYC Open Data makes the wealth of public data generated by various New York City agencies and other City organizations available for public use. It is funded by the New York City government. This specific dataset was provided by the Department of City Planning.
* Many different politics/policies in NYC affect what type of data is collected. There are many different reasons why these estimates might not be an accurate representation of differences across boroughs including homelessness, lack of documentation, people over or underreporting their income, etc… 
* We dropped rows of unnecessary housing data that did not look at the rent burden. We also dropped unnecessary neighborhoods that had missing values such as the Staten Island Cemetery. We further cleaned the data by only looking at the median estimates by borough, and median percent estimates by borough and dropped all other columns.
* Since this is not personalized data, the people involved might not be fully aware that their data is being used and collected. However, since it is in aggregate form there should be no privacy or expectation concerns.
* The raw dataset can be found [here](https://drive.google.com/file/d/1mL4DjAYRaL2f184_WdoQd-u0ETNvYvgU/view?usp=sharing).

##### Limitiations
* This data shows five year estimates from 2008-2012, so we don't get a glimpse of what is happening year to year and how the rent burden is changing for each district -- it might be getting better or worse for some districts and we cannot exactly tell from the data. Furthermore, this data is quite old and does not take into acccount recent political events and policies that likely have afffected housing policy in NYC.

### Dataset Two: [Poverty](https://data.cityofnewyork.us/City-Government/NYCgov-Poverty-Measure-Data-2018-/hveh-vctc).
##### Description
* The original dataset is fairly simple - it lists the poverty rate of each community district in NYC and also has rows for a summary of each of the boroughs.
* This dataset comes from NYC Open Data as well, as a part of an initiative to increase transparency, accessibility, and accountability of the City government and made possible through funding by the NYC government. This specific dataset came from the Mayor's Office for Econnomic Opportunity.
* Many different politics/policies in NYC affect what type of data is collected. There are many different reasons why these estimates might not be an accurate representation of differences across boroughs including homelessness, lack of documentation, people over or underreporting their income, etc… 
* To clean this dataset, we dropped the summary rows, created a new column to indicate what overall borough the community district was in, convert the percentages to floats for calculations, and sorted it alphabetically by community district.
* The NYCgov poverty rate is measured in a different way than the overall US poverty rate. According to their [website](https://www1.nyc.gov/site/opportunity/poverty-in-nyc/poverty-measure.page), the measure includes a threshold that accounts for the higher cost of housing in New York City and incorporates the value of programs intended to alleviate poverty; adjusting family incomes for benefits such as the Supplemental Nutritional Assistance Program (SNAP) and the Earned Income Tax Credit.
* Since this data is aggregated, there should be no privacy or expectation concerns for the individual.
* The raw dataset can be found [here](https://drive.google.com/file/d/1O_JcvgjGhp79fMZrFzXZX3dnw3ZlMXKL/view?usp=sharing).

##### Limitations
* This poverty measure only looks at a specific point in time - 2018 - so we cannot see how poverty levels have changed over time for the community districts which would definitely have an impact on our overall analysis.

### Dataset Three: [Population](https://data.cityofnewyork.us/City-Government/New-York-City-Population-By-Community-Districts/xi7c-iiu2)
##### Description
* The orginal dataset looked at population levels across community districts at different points in time -- 1970, 1980, 1990, 2000, and 2010, collected Census Bureaus' Decennial data dissemination. We cleaned our data to only look at the most recent measure of population (2010), that better aligns with our times of our other datasets.
* We had to create a new column that combined distict and community number to be able to merge with our other datsets.
* The data was provided by the Department of City Planning is owned by NYC Open Data.
* We used this data to provide a magnitude measurement for our levels of poverty and rent burden, so we have a better understanding of how many people are affected across districts since popoulation levels vary. 
* This data is not personalized and does not provide any sensitive information so there should be no privacy concerns.
* The raw dataset can be found [here](https://drive.google.com/file/d/1T_HYEXvHoydxVDyuijdM2dSu0P1zM-lZ/view?usp=sharing).

##### Limitations
* This measure of population is perpetually changing and being a decade old, is not an extremely accurate measure of population levels right now. However, it does align with our other datasets so it gives us a good look into the general trend.
* Census data itself always has its limitations -- people being missed, double reported, inaccurate responses. There is no way to completely know.

### Dataset Four: Student Temp Housing
##### Description
* For the finalized dataset for housing and education data, we combined two datasets. The rows represent different boroughs and school districts within New York City. The columns make up different demographics, including percentage of Asian, Black, Hispanic, White, and other races, along with percentage of students in poverty. Additionally, they show the total number of students at each public school regardless of housing situation, students residing in shelter, the number and percent of students in temporary housing. 
* Poverty is measured by students who qualify for a free or reduced price lunch program or those eligible for Human Resources Administration benefits. Students in temporary housing are defined as any student 21 or younger enrolled in school and experiencing housing instability at any point during the school year. Temporary housing includes sharing the housing of others, in shelter, in hotel/motel, or in other temporary housing. The total numbers of students and students in temporary housing are disaggregated by school, based on the last school or program a student attended. In 2017-18, there were 105,261 students residing in temporary housing at any point throughout the school year.  
* The total numbers of students and students in temporary housing are disaggregated by school, based on the last school or program a student attended. 
* The final, revised dataset was created in order to study relationships between districts and specific demographics in order to find insights into poverty levels and temporary housing numbers within the districts. 
* The data was collected using multiple data sources, including the DOE's Audited Register, biographic data from the Automate the Schools (ATS) system, and the Location Code Generation and Management System (LCGMS), and the Department of Homeless Services (DHS). Due to the occasional missing demographic data and long-term-absent students not being accounted for, some of the enrollment numbers do not add up. The people surveyed (students within New York City) are probably aware that their demographic and free lunch status is recorded but this information does not affect the data. 
* The original data set can be found [here](https://drive.google.com/drive/folders/1B8daIcWqa5voZ6YwI-Jy1ZA4WMcZJdzW?usp=sharing).

##### Limitations
* One significant limitation within the dataset is that only public schools are accounted for (tne school data accounts for charter schools while the housing data does not). The absense of private schools in the dataset results in inflated poverty, racial minority percentages, and housing data for each district as private schools historically account for a higher level of wealthier and white students due to the tuition and prep needed to admitted.
* Regarding the housing data, The Federal McKinney-Vento Homeless Assistance Act requires school districts to ask families to report on their housing situation. Schools are only required to provide the survey when they are aware that there is a change in address so there may be instances where schools have outdated information. Additionally, families might lie about the state of housing in their families in the questionnaires as they do not want schools prying into their family situation so as a result, the actual temporary housing numbers might be lower than they actually are. 

### Dataset Five: [School Quality](https://data.cityofnewyork.us/Education/2016-2017-School-Quality-Report-Elem-Middle-K-8-Sc/9n2i-9h9s)
##### Description 
* The original dataset looked at indicators of school quality that were both categorical and numerical. We decided to remove all categorical data and looked only at colummns that indicated percents for easier use.
* This data was acquired through a quality review, developed to assist New York City Department of Education (NYCDOE) schools in raising student achievement by looking behind a school’s performance statistics to ensure that the school is engaged in effective methods of accelerating student learning.
* New York City Department of Education owns this data.
* There might be privacy concerns as the only households/individuals that would be given to the school should be from those of students that attend those schools, but the data gives out no personally identifying/sensitive information.
* The raw dataset can be found [here](https://drive.google.com/file/d/1I0w3aqhiE0JePaylB1DQhxoUV4jwCT6J/view?usp=sharing).

#### Limitations
* This data only looks at traditional forms of education and does not take into account Head Start programs, private tutoring, and outside help, nor does it look at the educational attainment of parents that may have an effect on the academic outcomes of a student. 
* This data also does not look into economic factors of school success, such as teacher pay, funding for extracurricular activities that all have an effect on the learning environment. 